# Build and tune iPerf3.16 and above and test UDP performance in a single site

## Build a slice

In [ ]:
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# variable settings
slice_name = 'iPerf3 UDP slice'
sender_name = 'sender'
recver_name = 'recver'
model_name = 'NIC_ConnectX_6'
#site_override = 'STAR'
site_override = None
subnet = IPv4Network("192.168.1.0/24")

# iperf
iperf_url = 'https://github.com/esnet/iperf/releases/download/3.16/iperf-3.16.tar.gz'

In [ ]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

if not site_override:
    # need to have ConnectX6, PTP and be in continental US
    selected_site = fablib.get_random_site(filter_function=lambda x: x['nic_connectx_6_available'] >= 2 
                                              and x['location'][1] < lon_east
                                              and x['location'][1] > lon_west
                                              and x['ptp_capable'] is True) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

In [ ]:
# build a slice
slice = fablib.new_slice(name=slice_name)

node_attribs = {'site': selected_site,
                'cores': 32,
                'ram': 32,
                'disk': 100,
                'image': 'default_ubuntu_22',
               }

# create a network
net1 = slice.add_l2network(name=sender_name+'_'+recver_name+'_net', subnet=subnet)

nodes = {}
# create identical nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_addr in zip([sender_name, recver_name], subnet.hosts()):
    print(f'{node_name=} {node_addr=}')
    nodes[node_name] = slice.add_node(name=node_name, **node_attribs)
    nic_interface = nodes[node_name].add_component(model=model_name, name=node_name+'_nic').get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_addr)

# Submit the slice
slice.submit();

In [ ]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

## Run updates

In [ ]:
# run updates and install dependencies and reboot
commands = ['sudo apt-get -q update -y',
            'sudo apt-get -q install -y gcc make'
           ]

print('Updating...')
for node in slice.nodes:
    print(f'Working on {node.get_name()}')
    for command in commands:
        print(f'\tExecuting "{command}"')
        node.execute(command, output_file=node.get_name() + '_update.log', quiet=True)
        
print('Rebooting...')
reboot = 'sudo reboot'
for node in slice.nodes:
    node.execute(reboot)

slice.wait_ssh(timeout=360,interval=10,progress=True)
slice.update()
slice.test_ssh()
print("Reconnected!")

## Download, build and install iPerf3

In [ ]:
commands = ['curl -Ls ' + iperf_url + ' > iperf.tar.gz',
            'tar -zxf iperf.tar.gz',
            'sudo mkdir -p /opt && sudo chown ubuntu /opt',
            'cd iperf-3.16 && ./configure --prefix=/opt && make install',
           ]

for node in slice.nodes:
    print(f'Working on {node.get_name()}')
    for command in commands:
        print(f'\tExecuting "{command}"')
        stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        if not stderr and len(stderr) > 0:
            print(f'Error encountered with "{command}": {stderr}')

## Tune host performance

In [ ]:
slice = fablib.get_slice(slice_name)

for node in slice.get_nodes():
    nic_name = node.get_name() + '_nic'
    
    # Pin all vCPUs for VM to same Numa node as the component
    node.pin_cpu(component_name=nic_name)
    
    # User can also pass in the range of the vCPUs to be pinned 
    #node.pin_cpu(component_name=nic_name, cpu_range_to_pin="0-3")
    
    # Pin memory for VM to same Numa node as the components
    #node.numa_tune()
    
    # Reboot the VM
    node.os_reboot()

In [ ]:
# wait for reboot to complete and recover configuration
slice = fablib.get_slice(slice_name)

# Wait for the SSH Connectivity to be back
slice.wait_ssh()

In [ ]:
# Re-configuring the Network and set MTU
for node in slice.nodes:
    nic_name = node.get_name() + '_nic'
    intf = list(filter(lambda x: x.get_name()[-2:] == 'p1', node.get_component(name=nic_name).get_interfaces()))[0]
    command = f'sudo ip link set mtu 9000 dev {intf.get_physical_os_interface_name()}'
    print(f'Executing "{command}" on {node.get_name()}')
    node.config()
    stdout, stderr = node.execute(command, quiet=True)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')

## Test

## Manage Slice

### Extend

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()